In [21]:
import pandas as pd

"""
Reading the bert output
"""
bert_all = pd.read_csv("predictions-with-labels-all.csv", header=None)
bert_test = pd.read_csv("predictions-with-labels-test.csv", header=None)
bert_train = pd.read_csv("predictions-with-labels-train.csv", header=None)
bert_val = pd.read_csv("predictions-with-labels-val.csv", header=None)

"""
Reading the train, val , test data
"""
train = pd.read_csv("/Users/ItishaYadav1/CL-Team-Lab-Emotion-Classification/data/isear/isear-train.csv", on_bad_lines='skip', header=None)
print("Training data shape = ", train.shape)
val = pd.read_csv("/Users/ItishaYadav1/CL-Team-Lab-Emotion-Classification/data/isear/isear-val.csv", header=None)
print("Validation data shape = ", val.shape)
test = pd.read_csv("/Users/ItishaYadav1/CL-Team-Lab-Emotion-Classification/data/isear/isear-test.csv", header=None)
print("Test data shape = ", test.shape)

"""
applying assertion
"""
assert bert_all.shape[0] == (bert_train.shape[0] + bert_val.shape[0] + bert_test.shape[0])
bert_all.shape, bert_train.shape, bert_val.shape, bert_test.shape

Training data shape =  (5333, 2)
Validation data shape =  (1150, 2)
Test data shape =  (1146, 2)


((7619, 14), (5326, 14), (1148, 14), (1145, 14))

In [23]:
"""
Applying analysis on test
"""

# inp_data = pd.concat([test, val])
# bert_out = pd.concat([bert_test, bert_val])


inp_data = test
bert_out = bert_test

assert inp_data.shape[0] == bert_out.shape[0]

analysis_df = inp_data.merge(bert_out, left_index=True, right_index=True)
analysis_df = analysis_df[analysis_df["anger"]]



'\ninp_data = test\nbert_out = bert_test\n\nassert inp_data.shape[0] == bert_out.shape[0]\n\nanalysis_df = inp_data.merge(bert_out, left_index=True, right_index=True)\nanalysis_df = analysis_df[analysis_df["anger"]]\n\n'

In [24]:
bert_all

,0,1,2,3,4,5,6,7,8,9,10,11,12,13
0,-4.143373,-4.099311,-4.180825,-3.873905,3.270709,-3.883025,-3.976190,0.0,0.0,0.0,0.0,1.0,0.0,0.0
1,-4.454488,2.634034,-3.970481,-3.600777,-4.260800,-3.508614,-4.351669,0.0,1.0,0.0,0.0,0.0,0.0,0.0
2,-3.871844,-4.237987,-4.067472,-3.916490,3.443163,-4.022819,-3.743179,0.0,0.0,0.0,0.0,1.0,0.0,0.0
3,-4.045258,3.200204,-3.871938,-3.878467,-4.119332,-3.877209,-3.389095,0.0,1.0,0.0,0.0,0.0,0.0,0.0
4,-4.648178,-4.786387,-1.271109,-2.308218,-4.468682,0.195158,-2.507853,0.0,0.0,0.0,0.0,0.0,1.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7614,-3.077817,-5.027366,-3.306948,1.170537,-5.571788,-1.354663,-2.787492,0.0,0.0,0.0,1.0,0.0,0.0,0.0
7615,-4.497149,2.975088,-3.899056,-3.790592,-4.291361,-3.033495,-3.544418,0.0,1.0,0.0,0.0,0.0,0.0,0.0
7616,2.665590,-4.259548,-3.877426,-3.458135,-3.568334,-3.607715,-3.872411,1.0,0.0,0.0,0.0,0.0,0.0,0.0
7617,-4.251548,-4.527634,-3.848884,-4.058742,3.007751,-3.362574,-4.002860,0.0,0.0,0.0,0.0,1.0,0.0,0.0


In [ ]:


def mlp_classifier(X, y):
    from imblearn.over_sampling import SMOTE
    import pandas as pd
    from sklearn.model_selection import train_test_split
    """Perform PCA"""
    from sklearn.neighbors import KNeighborsClassifier
    from sklearn.decomposition import PCA
    from sklearn.discriminant_analysis import QuadraticDiscriminantAnalysis
    from sklearn.preprocessing import StandardScaler
    from sklearn.preprocessing import MinMaxScaler
    from sklearn.preprocessing import Normalizer

    oversample = SMOTE()

    X, y = oversample.fit_resample(X, y)
    print(y.value_counts())
    pca = PCA(n_components=3)
    reduced_data = pca.fit_transform(X)
    reduced_data_df = pd.DataFrame(reduced_data, columns=["dim1", "dim2", "dim3"])
    #reduced_data_df["1-2"] = X[1] - X[2]
    #reduced_data_df["2-3"] = X[2] -X[3]
    #reduced_data_df["3-4"] = X[3] -X[4]
    print(pca.explained_variance_ratio_)
    data_col = reduced_data_df.columns
    scalar = StandardScaler()
    #scalar = MinMaxScaler()
    scalar.fit(reduced_data_df)
    # transform data
    standard_data = scalar.transform(reduced_data_df)
    standard_df = pd.DataFrame(standard_data, columns=data_col)
    print(standard_df.shape)
    print(len(y))
    """
    Data Splitting
    """
    X_train, X_test, y_train, y_test = train_test_split(standard_df, y, test_size = 0.2,
                                                        random_state = 42, stratify = y)

    print(X_train.shape, y_train.shape, X_test.shape, y_test.shape)

    """Model training"""

    from sklearn.neural_network import MLPClassifier


    from sklearn.svm import SVC
    from sklearn.ensemble import RandomForestClassifier
    from numpy import loadtxt
    from xgboost import XGBClassifier
    # fit model no training data
    #clf = XGBClassifier()
    #\\MLP with random_state=1, max_iter=300, giving higest
    clf = MLPClassifier(max_iter=100, random_state=42, activation="tanh", solver="adam", hidden_layer_sizes=(100, ),
    learning_rate="adaptive")
    #clf = QuadraticDiscriminantAnalysis(store_covariance=True)
    #clf = SVC(kernel ='rbf', probability=True)
    clf.fit(X_train, y_train)

    #clf.fit(X_train, y_train)
    #clf= KNeighborsClassifier(n_neighbors=5, metric='minkowski', p=2 )
    #clf.fit(X_train, y_train)
    #model = RandomForestClassifier(max_depth=5, random_state=0)
    # parameter_space = {
    #     'hidden_layer_sizes': [(50,50,50), (50,100,50), (100,)],
    #     'activation': ['tanh', 'relu'],
    #     'solver': ['sgd', 'adam'],
    #     'alpha': [0.0001, 0.05],
    #     'learning_rate': ['constant','adaptive'],
    # }
    # from sklearn.model_selection import GridSearchCV
    #
    # clf = GridSearchCV(mlp, parameter_space, n_jobs=-1, cv=3)
    # clf.fit(X_train, y_train)
    #
    # # Best paramete set
    # print('Best parameters found:\n', clf.best_params_)
    #
    # # All results
    # means = clf.cv_results_['mean_test_score']
    # stds = clf.cv_results_['std_test_score']
    # for mean, std, params in zip(means, stds, clf.cv_results_['params']):
    #     print("%0.3f (+/-%0.03f) for %r" % (mean, std * 2, params))

    """Predictions"""
    from sklearn.metrics import accuracy_score
    from sklearn.metrics import classification_report
    from sklearn.metrics import confusion_matrix

    y_pred_train = clf.predict(standard_df)
    y_pred_train_proba = clf.predict_proba(standard_df)
    # evaluate predictions
    accuracy = accuracy_score(y, y_pred_train)
    print("Train Accuracy: %.2f%%" % (accuracy * 100.0))
    print(classification_report(y, y_pred_train))

    print(confusion_matrix(y, y_pred_train))
    print()
    print()

    # make predictions for test data
    y_pred = clf.predict(X_test)
    y_pred_prob = clf.predict_proba(X_test)

    top_df_test = X_test
    top_df_test["yhat"] = y_pred
    top_df_test[["y_pred_prob_1", "y_pred_prob_2"]] = y_pred_prob
    top_df_test["ytrue"] = y_test
    top_match = []
    for ind, row in top_df_test.iterrows():
        if row["ytrue"] == row["yhat"]:
            top_match.append(1)
        else:
            top_match.append(0)
    top_df_test["match"] = top_match
    # evaluate predictions
    accuracy = accuracy_score(y_test, y_pred)
    print("Test Accuracy: %.2f%%" % (accuracy * 100.0))
    print(classification_report(y_test, y_pred))

    print(confusion_matrix(y_test, y_pred))
    return top_df_test, pca, scalar, clf

"""
define y
"""
# transform the dataset
import pickle
a, pca, scalar, clf = mlp_classifier(X, match[0])
pickle.dump(pca, open("pca.p", 'wb'))
pickle.dump(scalar, open("scalar.p", 'wb'))
pickle.dump(clf, open("crisp_vs_soft_dm.p", 'wb'))

In [ ]:

"""
Strategy to approach the research question:
    1. Filter the dataset (all) to find misclassified examples.
    2. Find out the those specific sentences and see manually, if they are the potential examples of class overlapping.
    3. Train the ORD detector, and see of the correct label for missclassfied classes are in the top 3 recommendation or not?
    4. Evaluate the bert model, using our custom evaluator
"""


"""

As discussed with Prof, go with approach one and put approach 2 in future work
ORD:

1. BERT
2. Low_vs_high confidence detector

Soft decision maker:

1. Approach 1: Rule Based

    If overlapping region detected:

    Recommend top 2 emotions

    If non overlapping region detected:

    Recommend top emotion

2. Approach 2: Machine learning based
    1. Train Classifiers:
        1. Classifier 1 checks if top 2 can be recommended.
        2. Classifier 2 checks, if top 3 can be recommended.
        3. Worst case senario: model not confident with the emotion.
"""